<p style="font-weight:bold;"> <span style="font-size: 36px"> Reimport With Different Scope Test </span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Cash flow Import

Importing a file with N GICs and GRICs.

In [0]:
await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2020_12.csv")
    .WithFormat(ImportFormats.Cashflow)
    .WithTarget(DataSource)
    .WithActivityLog(Session, DataSource, DataSetReader)
    .ExecuteAsync()

In [0]:
var ifrsVars1 = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var dn1 = ifrsVars1.Select(x => x.DataNode).ToHashSet();
var dt11Prem1 = ifrsVars1.Where(x => x.DataNode == "DT1.1" && x.AmountType == "PR");
(ifrsVars1.Count(), dn1.Count(), dt11Prem1.Count())

# Cash flow Import - restatement
Importing a file for the same period as previous import but it only contains 1 GIC: DT1.1 where one amount type (Premium) has been removed

In [0]:
await Import.FromFile("Data/NominalCashflows_CH_2020_12_DT1.1NoPrem.csv")
    .WithFormat(ImportFormats.Cashflow)
    .WithTarget(DataSource)
    .WithActivityLog(Session, DataSource, DataSetReader)
    .ExecuteAsync()

In [0]:
var ifrsVars2 = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var dn2 = ifrsVars2.Select(x => x.DataNode).ToHashSet();
var dt11Prem2 = ifrsVars2.Where(x => x.DataNode == "DT1.1" && x.AmountType == "PR");
(ifrsVars2.Count(), dn2.Count(), dt11Prem2.Count())

# Test

## Import generate variables

In [0]:
ifrsVars1.Count().Should().NotBe(0);

In [0]:
ifrsVars2.Count().Should().NotBe(0);

## Count of data nodes does not change

In [0]:
dn2.Count().Should().Be(dn1.Count());

## Premiums are present with the first import and then deleted

In [0]:
dt11Prem1.Count().Should().NotBe(0);

In [0]:
dt11Prem2.Count().Should().Be(0);